In [1]:
import pandas as pd
import numpy as np

from typing import Dict, Any

from src.tvData import get_df, get_nn_mask, median_filter, get_dist

In [2]:
TRUE_DIST = 57.4

In [3]:
df = get_df(["02_dab"], "./data/run8/")
coords = df[["x", "y", "z"]].to_numpy()

In [4]:
no_filter = get_dist(coords=coords, min_step=0)
print("no post_processing, diff:\t", abs(no_filter.sum() - TRUE_DIST))

no post_processing, diff:	 462.9001598902963


In [5]:
best_params: Dict[str, Any] = {}

min_d = 10
for nn in range(10, 100, 5):
    for step in range(0, 40, 3):
        for kernel_size in range(3, 30, 2):
            nn_coords = coords[1: -1][get_nn_mask(df, threshold=nn)]
            med_coords = median_filter(nn_coords, kernel_size=kernel_size)
            dist_arr = get_dist(med_coords, min_step=step)
            dist = dist_arr.sum()
            diff = abs(dist - TRUE_DIST)
            if diff < min_d:
                min_d = diff
                best_params = {
                    "nn" : nn,
                    "kernel_size" : kernel_size,
                    "step" : step,
                    "diff" : diff
                }

print(best_params)

{'nn': 90, 'kernel_size': 29, 'step': 9, 'diff': 0.0002143177617668357}


In [6]:
nn_coords = coords[1: -1][get_nn_mask(df, threshold=90)]
med_coords = median_filter(nn_coords, kernel_size=11)
dist_arr = get_dist(med_coords, min_step=15)
dist = dist_arr.sum()
diff = abs(dist - TRUE_DIST)
print(diff)

0.3235614520366923


In [7]:
coords_z = df[["x", "y"]]
best_params_without_z: Dict[str, Any] = {}

min_d = 10
for nn in range(10, 120, 5):
    for step in range(0, 40, 3):
        for kernel_size in range(3, 30, 2):
            nn_coords = coords_z[1: -1][get_nn_mask(df, threshold=nn)]
            med_coords = median_filter(nn_coords, kernel_size=kernel_size)
            dist_arr = get_dist(med_coords, min_step=step)
            dist = dist_arr.sum()
            diff = abs(dist - TRUE_DIST)
            if diff < min_d:
                min_d = diff
                best_params_without_z = {
                    "nn" : nn,
                    "kernel_size" : kernel_size,
                    "step" : step,
                    "diff" : diff
                }

print(best_params_without_z)

{'nn': 30, 'kernel_size': 5, 'step': 9, 'diff': 0.0028177597150218503}
